In [13]:
import os

def make_fw_bin(lines:list[str], line_prefix, save_path):
    '''
        get the firs line that starts with the given prefix
        and get the hex string from it
        write it to a bin file
    '''
    
    for line in lines:
        if line.startswith(line_prefix):
            str_hex = line.replace(line_prefix, '').strip()
            
            # Convert hex string to bytes
            bytes_hex = bytes.fromhex(str_hex)
            # print(f'line_hex: {" ".join(f"{byte:02x}" for byte in bytes_hex)}')

            with open(save_path, 'wb') as f:
                f.write(bytes_hex)

            print(f'Wrote {len(bytes_hex)} bytes to {save_path}')
            
            return len(bytes_hex)



In [14]:
# V1  
la_data_path = '../la_data/touch_boot.txt'
with open(la_data_path, 'r') as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines if line.strip()]

output_dir = './output/ip7_touch_firmware'
save_name = 'ip7_touch_fw{}.bin'

# save_path = 'fw.bin'
            
line_prefix = [
    '33607258-36152228 SPI: MOSI transfer:',
    '36156813-36892515 SPI: MOSI transfer:',
    '36897678-36944066 SPI: MOSI transfer:',
    '36948434-36994162 SPI: MOSI transfer:',
    '36998385-37072273 SPI: MOSI transfer:',
    '37075121-37102741 SPI: MOSI transfer:',
]


os.makedirs(output_dir, exist_ok=True)
for i in range(len(line_prefix)):
    line_prefix_i = line_prefix[i]
    save_path = os.path.join(output_dir, f'ip7_touch_fw_{i}.bin')
    make_fw_bin(lines, line_prefix_i, save_path)    
    
    


Wrote 59248 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_0.bin
Wrote 17076 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_1.bin
Wrote 1040 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_2.bin
Wrote 1008 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_3.bin
Wrote 1492 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_4.bin
Wrote 224 bytes to ./output/ip7_touch_firmware\ip7_touch_fw_5.bin


In [39]:
# V2 make all MOSI transfers

match_word = 'SPI: MOSI transfer:'
exclude_words = [
    '0-33102006 SPI: MOSI transfer:',
    '33116420-33551371 SPI: MOSI transfer:',
    '33552614-33553032 SPI: MOSI transfer:',
]

la_data_path = '../la_data/touch_boot.txt'
with open(la_data_path, 'r') as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines if line.strip()]
    
    
    lines = [l for l in lines if match_word in l]
    print(f'Found {len(lines)} lines with "{match_word}" before excluding words.')
    
    lines = [l for l in lines if not any(ew in l for ew in exclude_words)]
    print(f'Found {len(lines)} lines with "{match_word}" after excluding words.')
        


# output_dir = './output/ip7_touch_firmware'
# save_name = 'ip7_touch_fw{}.bin'

save_path = './output/ip7_touch_fw_V2.txt'

with open(save_path, 'w') as f:
    for i, l in enumerate(lines):
        l.split(match_word)[1].strip()
        # to bytes
        bytes_hex = bytes.fromhex(l.split(match_word)[1].strip())
        
        byte_str = " ".join(f"{byte:02x}" for byte in bytes_hex)
        byte_str_0x =  ", ".join(f"0x{byte:02x}" for byte in bytes_hex)
        print(f'{len(bytes_hex)} bytes, {byte_str}')
        
        # format {len, {0x00, 0x01, ...}},
        
        f.write(f'{{{len(bytes_hex)}, (uint8_t[]){{{byte_str_0x}}}}},\n')
    
    
    
    
    





Found 49 lines with "SPI: MOSI transfer:" before excluding words.
Found 46 lines with "SPI: MOSI transfer:" after excluding words.
4 bytes, 1a a1 18 e1
59248 bytes, 18 e1 30 01 39 d8 00 00 00 00 01 11 f0 18 e5 9f f0 18 e5 9f f0 18 e5 9f f0 18 e5 9f f0 18 e5 9f 00 00 e1 a0 ff f0 e5 1f f0 18 e5 9f dc 60 00 00 00 40 00 00 dd 44 00 00 00 44 00 00 00 48 00 00 00 00 00 00 00 4c 00 00 b5 80 00 00 ff fe ea ff ff fe ea ff ff fe ea ff ff fe ea ff ff fe ea ff 00 00 ea 00 35 b2 ea 00 00 28 e2 8f 0c 00 e8 90 a0 00 e0 8a b0 00 e0 8b 70 01 e2 4a 00 0b e1 5a 35 ab 0a 00 00 0f e8 ba e0 14 e2 4f 00 01 e3 13 f0 03 10 47 ff 13 e1 2f 53 e0 00 40 54 10 00 40 c0 01 e2 8f ff 1c e1 2f 18 8a 78 03 30 01 07 5c 0f 64 d1 01 78 04 30 01 11 1d d1 01 78 05 30 01 3c 01 d0 05 78 06 30 01 70 0e 31 01 3c 01 d1 f9 07 1e d4 06 1c 2e d0 0e 3e 01 55 8c d1 fc 19 49 e0 09 78 04 30 01 1b 0c 35 01 78 26 34 01 70 0e 31 01 3d 01 d5 f9 42 91 d3 d7 47 70 20 10 e2 52 00 78 28 b0 00 78 28 a1 ff fb 8a ff 2e 82 e1 b0 00 30 28 b0 00 30 2